In [1]:
import os

mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-7.1.0-posix-seh-rt_v5-rev1\\mingw64\\bin'

os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
plotly.offline.init_notebook_mode(connected=True)
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from pandas.tseries.offsets import *
%matplotlib inline

from datetime import datetime

# 1.特徴量作成

In [3]:
log = pd.read_csv('log.tsv',delimiter="\t")
events = pd.read_csv('events.tsv', delimiter="\t")
users = pd.read_csv('users.tsv', delimiter="\t")
os.chdir('..')

In [4]:
print(log.shape)
log.head()

(1452089, 7)


,user_id,event_id,time_stamp,action_type,num_of_people,payment_method,total_price
0,1,6261,2015-03-19 09:15:50,3,1.0,クレカ,4000.0
1,1,127600,2017-06-05 12:33:17,1,NaN,NaN,NaN
2,1,127600,2017-06-05 12:37:09,1,NaN,NaN,NaN
3,1,125296,2017-06-06 14:01:27,1,NaN,NaN,NaN
4,1,125128,2017-06-06 14:03:57,1,NaN,NaN,NaN


In [5]:
print(events.shape)
events.head()

(158392, 11)


,event_id,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_start_at,prefecture,first_published_at,female_price,male_price,interest
0,1,20,99.0,25,44.0,2015-03-08 18:00:00,宮城県,NaN,1500.0,5000.0,NaN
1,2,30,99.0,30,49.0,2015-03-01 15:00:00,富山県,NaN,1000.0,6500.0,NaN
2,3,22,99.0,24,39.0,2015-03-07 19:00:00,富山県,NaN,1000.0,6000.0,NaN
3,4,20,99.0,25,44.0,2015-03-08 18:00:00,新潟県,NaN,1000.0,6000.0,NaN
4,5,20,99.0,20,39.0,2015-03-14 19:00:00,新潟県,NaN,1000.0,6000.0,NaN


In [6]:
print(users.shape)
users.head()

(69667, 5)


,user_id,age,gender,prefecture,created_on
0,1,34,女性,静岡県,2012-06-26
1,2,31,男性,鳥取県,2012-07-12
2,3,32,男性,東京都,2012-07-15
3,4,30,男性,東京都,2012-07-17
4,5,29,女性,埼玉県,2012-07-17


データ連結

In [7]:
merge1 = pd.merge(log,events, on ="event_id") 

In [8]:
merge2 = pd.merge(merge1,users, on ="user_id")

In [9]:
print(merge2.shape)
merge2.head()

(1450405, 21)


,user_id,event_id,time_stamp,action_type,num_of_people,payment_method,total_price,female_age_lower,female_age_upper,male_age_lower,...,event_start_at,prefecture_x,first_published_at,female_price,male_price,interest,age,gender,prefecture_y,created_on
0,1,6261,2015-03-19 09:15:50,3,1.0,クレカ,4000.0,27,39.0,27,...,2015-03-22 14:00:00,東京都,NaN,4000.0,6500.0,NaN,34,女性,静岡県,2012-06-26
1,1,127600,2017-06-05 12:33:17,1,NaN,NaN,NaN,24,36.0,26,...,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26
2,1,127600,2017-06-05 12:37:09,1,NaN,NaN,NaN,24,36.0,26,...,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26
3,1,127600,2017-06-05 12:32:04,1,NaN,NaN,NaN,24,36.0,26,...,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26
4,1,127600,2017-06-06 09:37:45,1,NaN,NaN,NaN,24,36.0,26,...,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26


# 特徴量を数値化

### 時間を数値化

In [10]:
merge2["time_stamp"] = pd.to_datetime(merge2["time_stamp"])

In [11]:
merge2["event_start_at"] = pd.to_datetime(merge2["event_start_at"])

In [12]:
merge2["first_published_at"] = pd.to_datetime(merge2["first_published_at"])

In [13]:
merge2["created_on"] = pd.to_datetime(merge2["created_on"])

In [14]:
print(merge2.shape)
merge2.head()

(1450405, 21)


,user_id,event_id,time_stamp,action_type,num_of_people,payment_method,total_price,female_age_lower,female_age_upper,male_age_lower,...,event_start_at,prefecture_x,first_published_at,female_price,male_price,interest,age,gender,prefecture_y,created_on
0,1,6261,2015-03-19 09:15:50,3,1.0,クレカ,4000.0,27,39.0,27,...,2015-03-22 14:00:00,東京都,NaT,4000.0,6500.0,NaN,34,女性,静岡県,2012-06-26
1,1,127600,2017-06-05 12:33:17,1,NaN,NaN,NaN,24,36.0,26,...,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26
2,1,127600,2017-06-05 12:37:09,1,NaN,NaN,NaN,24,36.0,26,...,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26
3,1,127600,2017-06-05 12:32:04,1,NaN,NaN,NaN,24,36.0,26,...,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26
4,1,127600,2017-06-06 09:37:45,1,NaN,NaN,NaN,24,36.0,26,...,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26


### データを数値にする

下の処理に約3分

In [18]:
merge2 = merge2.replace("クレカ", 0)
merge2 = merge2.replace("コンビニ", 1)
merge2 = merge2.replace("eマネー", 2)
print(merge2.shape)
merge2.head()

(1450405, 21)


,user_id,event_id,time_stamp,action_type,num_of_people,payment_method,total_price,female_age_lower,female_age_upper,male_age_lower,...,event_start_at,prefecture_x,first_published_at,female_price,male_price,interest,age,gender,prefecture_y,created_on
0,1,6261,2015-03-19 09:15:50,3,1.0,0,4000.0,27,39.0,27,...,2015-03-22 14:00:00,東京都,NaT,4000.0,6500.0,NaN,34,1,静岡県,2012-06-26
1,1,127600,2017-06-05 12:33:17,1,NaN,NaN,NaN,24,36.0,26,...,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,1,静岡県,2012-06-26
2,1,127600,2017-06-05 12:37:09,1,NaN,NaN,NaN,24,36.0,26,...,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,1,静岡県,2012-06-26
3,1,127600,2017-06-05 12:32:04,1,NaN,NaN,NaN,24,36.0,26,...,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,1,静岡県,2012-06-26
4,1,127600,2017-06-06 09:37:45,1,NaN,NaN,NaN,24,36.0,26,...,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,1,静岡県,2012-06-26


下の処理に約2分

In [16]:
merge2 = merge2.replace("女性", 1)
merge2 = merge2.replace("男性", 2)
print(merge2.shape)
merge2.head()

(1450405, 21)


,user_id,event_id,time_stamp,action_type,num_of_people,payment_method,total_price,female_age_lower,female_age_upper,male_age_lower,...,event_start_at,prefecture_x,first_published_at,female_price,male_price,interest,age,gender,prefecture_y,created_on
0,1,6261,2015-03-19 09:15:50,3,1.0,クレカ,4000.0,27,39.0,27,...,2015-03-22 14:00:00,東京都,NaT,4000.0,6500.0,NaN,34,1,静岡県,2012-06-26
1,1,127600,2017-06-05 12:33:17,1,NaN,NaN,NaN,24,36.0,26,...,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,1,静岡県,2012-06-26
2,1,127600,2017-06-05 12:37:09,1,NaN,NaN,NaN,24,36.0,26,...,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,1,静岡県,2012-06-26
3,1,127600,2017-06-05 12:32:04,1,NaN,NaN,NaN,24,36.0,26,...,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,1,静岡県,2012-06-26
4,1,127600,2017-06-06 09:37:45,1,NaN,NaN,NaN,24,36.0,26,...,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,1,静岡県,2012-06-26


In [ ]:
merge2 = merge2.replace("北海道", 1)
merge2 = merge2.replace("青森県", 2)
merge2 = merge2.replace("岩手県", 3)
merge2 = merge2.replace("宮城県", 4)
merge2 = merge2.replace("秋田県", 5)
merge2 = merge2.replace("山形県", 6)
merge2 = merge2.replace("福島県", 7)
merge2 = merge2.replace("茨城県", 8)
merge2 = merge2.replace("栃木県", 9)
merge2 = merge2.replace("群馬県", 10)
merge2 = merge2.replace("埼玉県", 11)
merge2 = merge2.replace("千葉県", 12)
merge2 = merge2.replace("東京都", 13)
merge2 = merge2.replace("神奈川県", 14)
merge2 = merge2.replace("新潟県", 15)
merge2 = merge2.replace("富山県", 16)
merge2 = merge2.replace("石川県", 17)
merge2 = merge2.replace("福井県", 18)
merge2 = merge2.replace("山梨県", 19)
merge2 = merge2.replace("長野県", 20)
merge2 = merge2.replace("岐阜県", 21)
merge2 = merge2.replace("静岡県", 22)
merge2 = merge2.replace("愛知県", 23)
merge2 = merge2.replace("三重県", 24)
merge2 = merge2.replace("滋賀県", 25)
merge2 = merge2.replace("京都府", 26)
merge2 = merge2.replace("大阪府", 27)
merge2 = merge2.replace("兵庫県", 28)
merge2 = merge2.replace("奈良県", 29)
merge2 = merge2.replace("和歌山県", 30)
merge2 = merge2.replace("鳥取県", 31)
merge2 = merge2.replace("島根県", 32)
merge2 = merge2.replace("岡山県", 33)
merge2 = merge2.replace("広島県", 34)
merge2 = merge2.replace("山口県", 35)
merge2 = merge2.replace("徳島県", 36)
merge2 = merge2.replace("香川県", 37)
merge2 = merge2.replace("愛媛県", 38)
merge2 = merge2.replace("高知県", 39)
merge2 = merge2.replace("福岡県", 40)
merge2 = merge2.replace("佐賀県", 41)
merge2 = merge2.replace("長崎県", 42)
merge2 = merge2.replace("熊本県", 43)
merge2 = merge2.replace("大分県", 44)
merge2 = merge2.replace("宮城県", 45)
merge2 = merge2.replace("鹿児島県", 46)
merge2 = merge2.replace("沖縄県", 47)

In [17]:
print(merge2.shape)
merge2.head()

(1450405, 21)


,user_id,event_id,time_stamp,action_type,num_of_people,payment_method,total_price,female_age_lower,female_age_upper,male_age_lower,...,event_start_at,prefecture_x,first_published_at,female_price,male_price,interest,age,gender,prefecture_y,created_on
0,1,6261,2015-03-19 09:15:50,3,1.0,クレカ,4000.0,27,39.0,27,...,2015-03-22 14:00:00,東京都,NaT,4000.0,6500.0,NaN,34,1,静岡県,2012-06-26
1,1,127600,2017-06-05 12:33:17,1,NaN,NaN,NaN,24,36.0,26,...,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,1,静岡県,2012-06-26
2,1,127600,2017-06-05 12:37:09,1,NaN,NaN,NaN,24,36.0,26,...,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,1,静岡県,2012-06-26
3,1,127600,2017-06-05 12:32:04,1,NaN,NaN,NaN,24,36.0,26,...,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,1,静岡県,2012-06-26
4,1,127600,2017-06-06 09:37:45,1,NaN,NaN,NaN,24,36.0,26,...,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,1,静岡県,2012-06-26


In [21]:
len(merge2)

1450405

In [27]:
target = pd.DataFrame(np.zeros(len(merge2)),columns = ["target"])

In [28]:
print(target.shape)
target.head()

(1450405, 1)


,target
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [30]:
type(merge2)

pandas.core.frame.DataFrame

In [31]:
type(target)

pandas.core.frame.DataFrame

In [51]:
merge3 = pd.concat([merge2,target],axis=1)

In [52]:
merge3.shape

(1450405, 22)

### モデリング

特徴量表示

In [53]:
merge3.columns

Index(['user_id', 'event_id', 'time_stamp', 'action_type', 'num_of_people',
       'payment_method', 'total_price', 'female_age_lower', 'female_age_upper',
       'male_age_lower', 'male_age_upper', 'event_start_at', 'prefecture_x',
       'first_published_at', 'female_price', 'male_price', 'interest', 'age',
       'gender', 'prefecture_y', 'created_on', 'target'],
      dtype='object')

In [54]:
feature_name = ['user_id', 'event_id', 'age','gender']
target_name = ["target"]

In [62]:
from sklearn.model_selection import train_test_split
X_train, X_valid,y_train, y_valid = train_test_split(
    merge3[feature_name], merge3[target_name], test_size=0.3, random_state=0)

In [63]:
param_grid = {'learning_rate':[0.5,0.35,0.1], 'max_depth':[5,7,10], 'subsample':[0.8,0.9,0.95],'n_estimators':[50,100,200],'colsample_bytree':[0.5,0.75,1.0]}
reg = xgb.XGBRegressor()
reg_cv = GridSearchCV(reg,param_grid,cv=10,scoring= 'neg_mean_squared_error',n_jobs=1)
reg_cv.fit(X_train,y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'learning_rate': [0.5, 0.35, 0.1], 'max_depth': [5, 7, 10], 'subsample': [0.8, 0.9, 0.95], 'n_estimators': [50, 100, 200], 'colsample_bytree': [0.5, 0.75, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

In [64]:
reg = xgb.XGBRegressor(**reg_cv.best_params_)
eval_set = [(X_valid, y_valid)]
reg.fit(X_train,y_train,early_stopping_rounds=50,eval_metric='rmse',eval_set=eval_set,verbose=True)

[0]	validation_0-rmse:0.25
Will train until validation_0-rmse hasn't improved in 50 rounds.
[1]	validation_0-rmse:0.125
[2]	validation_0-rmse:0.0625
[3]	validation_0-rmse:0.03125
[4]	validation_0-rmse:0.015625
[5]	validation_0-rmse:0.007813
[6]	validation_0-rmse:0.003906
[7]	validation_0-rmse:0.001953
[8]	validation_0-rmse:0.000977
[9]	validation_0-rmse:0.000488
[10]	validation_0-rmse:0.000244
[11]	validation_0-rmse:0.000122
[12]	validation_0-rmse:6.1e-05
[13]	validation_0-rmse:3.1e-05
[14]	validation_0-rmse:1.5e-05
[15]	validation_0-rmse:8e-06
[16]	validation_0-rmse:4e-06
[17]	validation_0-rmse:2e-06
[18]	validation_0-rmse:1e-06
[19]	validation_0-rmse:0
[20]	validation_0-rmse:0
[21]	validation_0-rmse:0
[22]	validation_0-rmse:0
[23]	validation_0-rmse:0
[24]	validation_0-rmse:0
[25]	validation_0-rmse:0
[26]	validation_0-rmse:0
[27]	validation_0-rmse:0
[28]	validation_0-rmse:0
[29]	validation_0-rmse:0
[30]	validation_0-rmse:0
[31]	validation_0-rmse:0
[32]	validation_0-rmse:0
[33]	validat

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.5, gamma=0, learning_rate=0.5, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=50,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)

In [1]:
importances = pd.Series(reg.feature_importances_, index = X_train.columns)
importances = importances.sort_values()
importances.plot(kind = "barh")
plt.title("feature imporance")
plt.show()

NameError: name 'pd' is not defined